# More On Word Embeddings

- effect of epoch
- efect of windows
- effect of dim



class gensim.models.word2vec.Word2Vec(sentences=None, corpus_file=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), max_final_vocab=None)¶

In [1]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_text, download_tfds_imdb_as_text_tiny
from lib.word_emb import run_pipeline, train_or_load_wv, train_or_load_wv_transfer




SyntaxError: invalid syntax (word_emb.py, line 110)

In [11]:
dataset  = download_tfds_imdb_as_text() # tuple of (X_train, X_test, y_train, y_test)
tiny_dataset = download_tfds_imdb_as_text_tiny() # first 100 samples from dataset

PRETRAINED_WV_MODEL_PATH = "./GoogleNews-vectors-negative300.bin"

# Dafault

dim 300
windows 5
iter 5
skip gram
minvount 1

In [6]:
model_train = train_or_load_wv(dataset[0]) 
_, vectorizer = run_pipeline(dataset, model_train)

Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.87 2
F1 on test set: 0.86


```
Training Word2Vec...
Loading tokenized document from disk...
Finished loading tokenized document in 0.37s!
Finished training Word2Vec and saved to disk!
Loading tokenized document from disk...
Finished loading tokenized document in 0.37s!
Loading tokenized document from disk...
Finished loading tokenized document in 0.36s!
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.87
F1 on test set: 0.86
```

# Windows

In this experiment, 

In [5]:
# approximate running time: 12 mins

model_train_window_15 = train_or_load_wv(dataset[0],window=15)
_, _ = run_pipeline(dataset, model_train_window_15)

model_train_window_30 = train_or_load_wv(dataset[0], window=30)
_, _ = run_pipeline(dataset, model_train_window_30)


Load Word2Vec from disk!
Loading tokenized document from disk...
Finished loading tokenized document in 0.37s!
Loading tokenized document from disk...
Finished loading tokenized document in 0.35s!
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
Load Word2Vec from disk!
Loading tokenized document from disk...
Finished loading tokenized document in 0.37s!
Loading tokenized document from disk...
Finished loading tokenized document in 0.36s!
Best parameters set found on development set:  {'C': 10}
Best F1 on development set: 0.89
F1 on test set: 0.88
681.8944387435913


```
Load Word2Vec from disk!
Loading tokenized document from disk...
Finished loading tokenized document in 0.37s!
Loading tokenized document from disk...
Finished loading tokenized document in 0.35s!
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
Load Word2Vec from disk!
Loading tokenized document from disk...
Finished loading tokenized document in 0.37s!
Loading tokenized document from disk...
Finished loading tokenized document in 0.36s!
Best parameters set found on development set:  {'C': 10}
Best F1 on development set: 0.89
F1 on test set: 0.88
681.8944387435913
```

# iter

In [7]:
# approximate running time: 40 mins
import time

now = time.time()

model_train_iter_2 = train_or_load_wv(dataset[0], iter=2, window=15)
_, _ = run_pipeline(dataset, model_train_iter_2)

model_train_iter_10 = train_or_load_wv(dataset[0],iter=10, window=15)
_, _ = run_pipeline(dataset, model_train_iter_10)

model_train_iter_15 = train_or_load_wv(dataset[0], iter=30, window=15)
_, _ = run_pipeline(dataset, model_train_iter_15)


print(time.time() - now)

Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.88 2
F1 on test set: 0.87
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89 2
F1 on test set: 0.87
Best parameters set found on development set:  {'C': 10}
Best F1 on development set: 0.89 2
F1 on test set: 0.87
3352.4832718372345


# dim 

```
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.88 2
F1 on test set: 0.87
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89 2
F1 on test set: 0.87
Best parameters set found on development set:  {'C': 10}
Best F1 on development set: 0.89 2
F1 on test set: 0.87
3352.4832718372345
```

In [8]:
# approximate running time: 40 mins
import time

now = time.time()

model_train_dim_100 = train_or_load_wv(dataset[0], size=100, window=15)
_, _ = run_pipeline(dataset, model_train_dim_100)

model_train_dim_500 = train_or_load_wv(dataset[0], size=500, window=15)
_, _ = run_pipeline(dataset, model_train_dim_500)

model_train_dim_1000 = train_or_load_wv(dataset[0], size=1000, window=15)
_, _ = run_pipeline(dataset, model_train_dim_1000)


print(time.time() - now)

Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.88
F1 on test set: 0.87
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
3940.248155117035


# transfer
- lock
- not lock
- limit vocab

In [ ]:
import time
now = time.time()
model_transfer_lock = train_or_load_wv_transfer(dataset[0], PRETRAINED_WV_MODEL_PATH)
_, _ = run_pipeline(dataset, model_transfer_lock)

model_transfer_not_lock = train_or_load_wv_transfer(dataset
                                                    [0], PRETRAINED_WV_MODEL_PATH, lockf=1)
_, _ = run_pipeline(dataset, model_transfer_not_lock)

print(time.time() - now)

```
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.85 2
F1 on test set: 0.85
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.85 2
F1 on test set: 0.85
3211.6761479377747
```

In [3]:
import time
now = time.time()
model_transfer_lock = train_or_load_wv_transfer(dataset[0], PRETRAINED_WV_MODEL_PATH, iter=2, window=15)
_, dense_lock = run_pipeline(dataset, model_transfer_lock)
dense_lock.print_stat()
# model_transfer_not_lock = train_or_load_wv_transfer(dataset[0], PRETRAINED_WV_MODEL_PATH, lockf=1, iter=2)
# _, dense_unlock = run_pipeline(dataset, model_transfer_not_lock)

print(time.time() - now)

Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.86
F1 on test set: 0.85
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.005682488993566625
%unk in vocab 0.010098350178710915
2014.812409877777


In [4]:
import time
now = time.time()
model_transfer_lock = train_or_load_wv_transfer(dataset[0], PRETRAINED_WV_MODEL_PATH, iter=5, window=15)
_, dense_lock = run_pipeline(dataset, model_transfer_lock)
dense_lock.print_stat()
model_transfer_not_lock = train_or_load_wv_transfer(dataset[0], PRETRAINED_WV_MODEL_PATH, lockf=1, iter=5,window=15)
_, dense_unlock = run_pipeline(dataset, model_transfer_not_lock)

print(time.time() - now)

Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.86
F1 on test set: 0.85
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.005682488993566625
%unk in vocab 0.010098350178710915
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.86
F1 on test set: 0.85
2545.27494764328


In [7]:
dense_unlock.print_stat()

fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.005682488993566625
%unk in vocab 0.010098350178710915


# top N

In [13]:
from gensim.models import Word2Vec, KeyedVectors
pretrained_word2vec = KeyedVectors.load_word2vec_format(PRETRAINED_WV_MODEL_PATH, binary=True)

In [15]:
list(pretrained_word2vec.vocab)[:10]

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']

In [ ]:
import time
now = time.time()
model_transfer_n_transfer_100k = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    iter=5,
    window=15,
    n_transfer=100000
)
_, dense_unlock = run_pipeline(dataset, model_transfer_n_transfer_100k)

print(time.time() - now)